# Data Memo

1. [Overview of Dataset](#overview-of-dataset)
1. [Overview of Research Question](#overview-of-research-question)
1. [Proposed Timeline](#proposed-timeline)
1. [Questions or Concerns](#questions-or-concerns)

### Overview of Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Before I answer any of the required questions, I will first import my data I have decided on so far. Everything comes from Kaggle at the following link: https://www.kaggle.com/datasets/open-source-sports/baseball-databank?resource=download. So far I have decided on using the Master.csv, Salaries.csv, and Batting.csv files which all have data from 1871 through 2015, but I will only be working with data from year 2000 to 2015. The Master.csv file contains personal information of all the players in the dataset, but I will only be using this to gather the names of players to make importing any future data easier since currently my datasets rely on player IDs. The Salaries.csv file contains team and league information, as well as the salary players have for any given year. Finally, my Batting.csv file contains 

In [50]:
names = pd.read_csv('Master.csv')
names['nameFull'] = names['nameFirst'] + ' ' + names['nameLast']
names = names[['playerID', 'nameFull']]

salaries = pd.read_csv('Salaries.csv')
salaries = salaries[salaries['yearID'] >= 2000]

batting_stats = pd.read_csv('Batting.csv')
batting_stats = batting_stats[batting_stats['yearID'] >= 2000]

merged_data = pd.merge(salaries, names, on='playerID', how='inner')

new_merged = pd.merge(batting_stats, merged_data, on=['playerID', 'yearID'], how='inner')

final_data = new_merged[['playerID', 'nameFull', 'yearID', 'salary', 'teamID_x', 'stint',
                        'lgID_x', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS',
                        'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP']]

***

### Overview of Research Question

***

### Proposed Timeline

***

### Questions or Concerns